In [ ]:
import pandas as pd

df = pd.read_csv('crop_recommendation.csv') #load the dataset
print(df.head()) #display the first few rows of the dataset
print(df.info()) #display information about the dataset


In [ ]:
df.isnull().sum() # Check for missing/null values in the dataset


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1) # Features (N, P, K, temperature, humidity, ph, rainfall)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split the dataset into training and testing sets


In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier() #high data accuracy
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred)) # Print accuracy of the model
print(classification_report(y_test, y_pred)) # Print classification report


In [ ]:
def recommend_crop(N, P, K, temperature, humidity, ph, rainfall): # Function to recommend crops based on input parameters
    input_data = pd.DataFrame([[N, P, K, temperature, humidity, ph, rainfall]],
                              columns=['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'])
    
    probs = model.predict_proba(input_data)[0] #
    top_5_indices = probs.argsort()[::-1][:5] # Get indices of top 5 crops
    top_5_crops = [(model.classes_[i], round(probs[i]*100, 2)) for i in top_5_indices] #get top 5 crops with their probabilities
    
    return top_5_crops


In [ ]:
import joblib
joblib.dump(model, 'crop_recommendation_model.pkl') # Save the trained model to a file for future use


In [ ]:
%pip install matplotlib #graphical representation of the data


In [69]:
# crop_recommend_cli.py

import pandas as pd
import joblib
import matplotlib.pyplot as plt

model = joblib.load('crop_recommendation_model.pkl')

def recommend_crop(N, P, K, temperature, humidity, ph, rainfall):
    input_data = pd.DataFrame([[N, P, K, temperature, humidity, ph, rainfall]],
                              columns=['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'])
    probs = model.predict_proba(input_data)[0] #get probabilities of each crop
    top_5_indices = probs.argsort()[::-1][:5] # Get indices of top 5 crops
    top_5_crops = [(model.classes_[i], round(probs[i]*100, 2)) for i in top_5_indices] #get top 5 crops with their probabilities
    return top_5_crops

def plot_recommendations(crops):
    labels = [crop for crop, _ in crops] 
    scores = [score for _, score in crops]

    plt.figure(figsize=(8, 5))
    bars = plt.barh(labels[::-1], scores[::-1], color='red',edgecolor='black')
    plt.xlabel("Suitability (%)")
    plt.title("Top 5 Recommended Crops")
    plt.tight_layout()

    # Add confidence % on bars
    for bar, score in zip(bars, scores[::-1]):
        plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                 f"{score}%", va='center')
    
    plt.show()

def main():
    print("\n🌾 Data-Driven Crop advisory system 🌾")
    try:
        N = float(input("Enter Nitrogen (N): "))
        P = float(input("Enter Phosphorous (P): "))
        K = float(input("Enter Potassium (K): "))
        temperature = float(input("Enter Temperature (°C): "))
        humidity = float(input("Enter Humidity (%): "))
        ph = float(input("Enter pH Level: "))
        rainfall = float(input("Enter Rainfall (mm): "))

        top_crops = recommend_crop(N, P, K, temperature, humidity, ph, rainfall)

        print("\nTop 5 Recommended Crops Based on Your Input:")
        for idx, (crop, _) in enumerate(top_crops, 1):
            print(f"{idx}. {crop}")

        plot_recommendations(top_crops)

    except Exception as e:
        print("\n❌ Error occurred:")
        print(e)

    input("\nPress Enter to exit...")

if __name__ == "__main__":
    main()
